In [ ]:
import os
import numpy as np 
import scipy.io.wavfile as wav 
import python_speech_features as psf
import hmmlearn.hmm as hmm

In [ ]:
def extract_mfcc_features(filename):
    # load the wave file and get the sample rate and audio data
    sample_rate, audio = wav.read(filename)
    # extract mfcc features
    mfcc_features = psf.mfcc(audio, samplerate=sample_rate)
    return mfcc_features

In [ ]:
extract_mfcc_features("/home/ix502iv/Documents/Audio_Trad/DTW/Lecture/audio/cat.wav")

In [ ]:
# load the .wav file for each word in the training set
word_directories = ["../Day_2/Gen/1/","../Day_2/Gen/2/","../Day_2/Gen/3/",
                    "../Day_2/Gen/4/", "../Day_2/Gen/5/"] # directories where each of the word is contained
word_models = [] # initialize an empty list where we will store trained models

for word_directory in word_directories:
    word_mfcc_features = []
    for filename in os.listdir(word_directory):
        if filename.endswith(".wav"):
            filepath = os.path.join(word_directory, filename)
            word_mfcc_features.append(extract_mfcc_features(filepath))

    # stack mfcc features for all audio files in the directory
    word_mfcc_features = np.vstack(word_mfcc_features)
    
    # train an HMM model on the MFCC features
    num_states = 5 # hidden number of states.
    num_mixtures = 13 # btn 8 and 16

    # specify the starting probs
    start_prob = np.array([0.1, 0.1, 0.1, 0.1, 0.6])

    word_model = hmm.GMMHMM(n_components=num_states, n_mix=num_mixtures, startprob_prior=start_prob)
    word_model.fit(word_mfcc_features)
    word_models.append(word_model)

In [ ]:
def validate_model(word_mfcc_features, word_models):
    log_likelihood = np.zeros(len(word_models))

    for i, model in enumerate(word_models):
        log_likelihood[i] = model.score(word_mfcc_features)

    # predict the word with highest likelihood
    word_index = np.argmax(log_likelihood)
    return word_index

In [ ]:
word_index= validate_model(word_mfcc_features, word_models)